In [ ]:
!pip install transformers
!pip install sentencepiece

import sys
import os
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import GPT2Tokenizer, TFGPT2Model
from transformers import DebertaTokenizer, DebertaForSequenceClassification
import torch
import re
from tqdm import tqdm
import pdb
import numpy as np
from scipy.linalg import eigh
import torch.nn as nn
softmax = nn.Softmax(dim=1)


def predict(question, model, tokenizer, device = 'cpu', num_beams = 10, num_return_sequences = 10, max_sequence_length = 32):
    """This function generates sequence = answer on given question

    1) it encodes qustion using tokenizer
    2) converts tokenized text to device
    3) generate answer with dredefined beam size using tokenized text
    4) as output we receive answers and their probabilities

    """

    input_ids = tokenizer([question], return_tensors="pt").input_ids
    input_ids = input_ids.to(device)

    out = model.generate(input_ids,
                         num_return_sequences = num_return_sequences,
                         num_beams = num_beams,
                         eos_token_id = tokenizer.eos_token_id,
                         pad_token_id = tokenizer.pad_token_id,
                         output_scores = True,
                         return_dict_in_generate=True,
                         early_stopping=True,
                         max_length=max_sequence_length)

    prediction = [tokenizer.decode(out.sequences[i], skip_special_tokens=True) for i in range(num_return_sequences)]


    return prediction


def compute_jaccard_similarity(lst):
    jaccard_sim_mat = np.zeros((len(lst), len(lst)))
    scores = []
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            set1 = set(lst[i].lower().split())
            set2 = set(lst[j].lower().split())
            intersection = len(set1 & set2)
            union = len(set1 | set2)
            #print(set1, set2, intersection, union,)
            jaccard_score = intersection / union

            jaccard_sim_mat[i, j] = jaccard_score
            #scores.append((lst[i], lst[j], jaccard_score))
    return jaccard_sim_mat#scores


def get_pairs(lst):
    pairs = []
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            pairs.append((lst[i], lst[j], i, j))
    return pairs

def get_pairs_semsets(lst):
    pairs = []
    for i in range(len(lst)-1):
            pairs.append((lst[i], lst[i+1]))
    return pairs

def U_Num_Sem_Sets(answers):

    lst = get_pairs_semsets(answers)
    num_sets = 1

    for (sentence_1, sentence_2) in lst:
        # Tokenize input sentences
        encoded_input_forward = tokenizer_nli(sentence_1, sentence_2, return_tensors='pt')
        encoded_input_backward = tokenizer_nli(sentence_2, sentence_1, return_tensors='pt')


        logits_forward = model_nli(**encoded_input_forward).logits.detach()
        logits_backward = model_nli(**encoded_input_backward).logits.detach()

        probs_forward = softmax(logits_forward)
        probs_backward = softmax(logits_backward)

        a_nli_entail_forward = probs_forward[0][2]
        a_nli_entail_backward = probs_backward[0][2]

        p_entail_forward = probs_forward[0][2]
        p_entail_backward = probs_backward[0][2]

        a_nli_contra_forward = 1 - probs_forward[0][0]
        a_nli_contra_backward = 1 - probs_backward[0][0]

        p_contra_forward = probs_forward[0][0]
        p_contra_backward = probs_backward[0][0]

        if (p_entail_forward > p_contra_forward) & (p_entail_backward > p_contra_backward):
            pass
        else:
            num_sets += 1

        return num_sets




def compute_nli_similarity(lst, order = "forward"):
    nli_sim_mat_entail, nli_sim_mat_contra = np.zeros((len(lst), len(lst))), np.zeros((len(lst), len(lst)))
    nli_prob_mat_entail, nli_prob_mat_contra = np.zeros((len(lst), len(lst))), np.zeros((len(lst), len(lst)))
    pairs = get_pairs(lst)
    result = []
    nli_prob_mat_entail = []
    nli_prob_mat_contra = []

    j = 1

    for i, (sentence_1, sentence_2) in enumerate(pairs):

        j=j%len(lst)
        print(i, j)

        if order == 'forward':
            encoded_input = tokenizer_nli(sentence_1, sentence_2, return_tensors='pt')
        elif order == 'backward':
            encoded_input = tokenizer_nli(sentence_2, sentence_1, return_tensors='pt')

        # Perform NLI classification
        logits = model_nli(**encoded_input).logits.detach()
        probs = softmax(logits)
        a_nli_entail = probs[0][2]
        p_entail = probs[0][2]

        a_nli_contra = 1 - probs[0][0]
        p_contra = probs[0][0]

        nli_sim_mat_entail[i, j] = a_nli_entail
        nli_sim_mat_contra[i, j] = a_nli_contra
        # nli_prob_mat_entail.append(p_entail.item())
        # nli_prob_mat_contra.append(p_contra.item())

        j+=1

           #print(sentence_1, sentence_2, a_nli_entail, a_nli_contra, '\n')

    return nli_sim_mat_entail, nli_sim_mat_contra, nli_prob_mat_entail, nli_prob_mat_contra

def compute_adjaency_mat(answers):
    W = np.eye(len(answers))
    pairs = get_pairs(answers)

    for (sentence_1, sentence_2, i ,j) in pairs:
        # Tokenize input sentences
        encoded_input_forward = tokenizer_nli(sentence_1, sentence_2, return_tensors='pt')
        encoded_input_backward = tokenizer_nli(sentence_2, sentence_1, return_tensors='pt')


        logits_forward = model_nli(**encoded_input_forward).logits.detach()
        logits_backward = model_nli(**encoded_input_backward).logits.detach()

        probs_forward = softmax(logits_forward)
        probs_backward = softmax(logits_backward)

        a_nli_entail_forward = probs_forward[0][2]
        a_nli_entail_backward = probs_backward[0][2]

        # p_entail_forward = probs_forward[0][2]
        # p_entail_backward = probs_backward[0][2]

        a_nli_contra_forward = 1 - probs_forward[0][0]
        a_nli_contra_backward = 1 - probs_backward[0][0]

        # p_contra_forward = probs_forward[0][0]
        # p_contra_backward = probs_backward[0][0]

        w = (a_nli_entail_forward + a_nli_entail_backward) / 2
        W[i, j] = w
        W[j, i] = w

    return W

def U_EigVal_Laplacian(answers):
    W = compute_adjaency_mat(answers)
    D = np.diag(W.sum(axis=1))
    D_inverse_sqrt = np.linalg.inv(np.sqrt(D))
    L = np.eye(D.shape[0]) - D_inverse_sqrt @ W @ D_inverse_sqrt
    return sum([max(0, 1 - lambda_k) for lambda_k in np.linalg.eig(L)[0]])


def U_DegMat(answers):
    #The Degree Matrix
    W = compute_adjaency_mat(answers)
    D = np.diag(W.sum(axis=1))
    return np.trace(len(answers) - D) / (len(answers) ** 2)

def U_Eccentricity(answers, k = 2):

    W = compute_adjaency_mat(answers)
    D = np.diag(W.sum(axis=1))
    D_inverse_sqrt = np.linalg.inv(np.sqrt(D))
    L = np.eye(D.shape[0]) - D_inverse_sqrt @ W @ D_inverse_sqrt

    # k is hyperparameter  - Number of smallest eigenvectors to retrieve
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = eigh(L)
    smallest_eigenvectors = eigenvectors[:, :k]
    V_mat = smallest_eigenvectors - smallest_eigenvectors.mean(axis = 0)


    norms = np.linalg.norm(V_mat, ord = 2, axis=0)
    U_Ecc = np.linalg.norm(norms, 2)
    C_Ecc_s_j = norms
    return U_Ecc, C_Ecc_s_j




# LLM
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-small-ssm-nq")
tokenizer = AutoTokenizer.from_pretrained("google/t5-small-ssm-nq")

# NLI model
model_name = "microsoft/deberta-large-mnli"
model_nli = DebertaForSequenceClassification.from_pretrained(model_name)
tokenizer_nli = DebertaTokenizer.from_pretrained(model_name)

# Questions
questions = ['What is the capital of Russia?',
             'When Albert Einstein was born?']

# Predictions
predictions  = [predict_and_proba(question = question, model = model, tokenizer = tokenizer, num_beams=30, num_return_sequences=10) for question in questions]


In [ ]:
# For example we have the following 3 answers (m = 3) of LLM for a Question - "What is the capital of Russia"
answers = ['Moscow', 'Olimpia, Greese', 'The capital of Russia is Moscow']

U_NumSets = U_Num_Sem_Sets(answers) # number of semantic groups in answer
U_EigV = np.round(U_EigVal_Laplacian(answers), 3) # (Due to the Theorem) A continuous analogue of number of semantic sets (higher = bogger uncertainty)
U_deg = np.round(U_DegMat(answers), 3) # average pairwise distance (less -> more confident because distance between answers is smaller). Since elems on diag of mat D are sums of similarities between the particular number (position in matrix) and other answers
U_Ecc = np.round(U_Eccentricity(answers)[0], 6) # frobenious norm (euclidian norm) between all eigen vectors that are informative embeddings of graph Laplacian (lower this value -> answers are closer in terms of euclidian distance between embeddings = eigenvectors)

print(f"Uncertaimty measures for answers {answers}: \n")
print(f"Number of Semantic Sets (U_NumSets) = {U_NumSets}")
print(f"Sum of Eigenvalues of the Graph Laplacian (U_EigV) = {U_EigV}")
print(f"The Degree Matrix (U_deg) = {U_deg}")
print(f"Eccentricity (U_Ecc) = {U_Ecc}")

In [ ]:
# For example we have the following 3 answers (m = 3) of LLM for a Question - "What is the capital of Russia"
answers = ['Moscow', 'Moscow, Russia', 'The capital of Russia is Moscow']

U_NumSets = U_Num_Sem_Sets(answers)
U_EigV = np.round(U_EigVal_Laplacian(answers), 3)
U_deg = np.round(U_DegMat(answers), 3)
U_Ecc = np.round(U_Eccentricity(answers)[0], 6)

print(f"Uncertaimty measures for answers {answers}: \n")
print(f"Number of Semantic Sets (U_NumSets) = {U_NumSets}")
print(f"Sum of Eigenvalues of the Graph Laplacian (U_EigV) = {U_EigV}")
print(f"The Degree Matrix (U_deg) = {U_deg}")
print(f"Eccentricity (U_Ecc) = {U_Ecc}")